# Advance Parameter Management

This is a very handy feature of `BqT` if you get used to it, the idea is to set parameters and magic numbers you have in your notebook in one place and forget about them.

## Basic Example

In [3]:
# add to sys.path
import sys

sys.path.insert(0, "../../")
sys.path.insert(0, "../../src")

from bqt import bqt

In [4]:
# set anything you want:
bqt.set_param("min_age", 18)
bqt.set_param("max_age", 60)

In [5]:
# you can now get them:
bqt.get_param("min_age")

18

In [7]:
# use them in a query:
query = "SELECT * FROM my_table WHERE age BETWEEN {min_age} AND {max_age}"
bqt.query(query)  # this will automatically replace them for you

### Dates
Dates are a bit of an special case, `BqT` parses them for you automatically:

In [4]:
bqt.set_param("my_birthday", "1910-12-01")

In [ ]:
# use them in a query:
query = "SELECT * FROM my_table WHERE date = '{my_birthday}'"
bqt.query(query)  # this will result in: SELECT * FROM my_table WHERE date = '1910-12-01'

# or change the format on the fly
query = "SELECT * FROM my_table_* WHERE _TABLE_SUFFIX = '{my_birthday_YYYYMMDD}'"
bqt.query(query)  # this will result in: SELECT * FROM my_table_* WHERE _TABLE_SUFFIX = '19101201'

### Date Offsets
There are many instance where you want to query a range, with `BqT` that as simple as:

In [ ]:
query = "SELECT * FROM my_table WHERE date BETWEEN '{my_birthday[-2Y]}' AND '{my_birthday[+2M+2W]}'"
bqt.query(query)  # this will result in: SELECT * FROM my_table WHERE date BETWEEN '1908-12-01' AND '1911-02-15'

For this to work, just put the offset in brackers (`[`, `]`) and put any of the modifiers below:
* `H`: hours
* `D`: days
* `W`: weeks
* `M`: months
* `Y`: years

For each modifier, use the format `+/-` then `Number` then the modifier, e.g.:
* `+2D`: plus two days
* `-6D+10H`: minus six days and plus 2 hours

## Advance Example
You can as easily specify date ranges, this are specially useful to create partitioned data right from your notebook

In [ ]:
for day in bqt.param_range("day", "2018-01-01", "2018-01-10"):
    bqt.create_table(
        # automatically replaced with dates from 2018-01-01 to 2018-01-10
        "SELECT * FROM my_table WHERE day = '{day}'",
        # also works in the table name, this will create one partition per day
        "my_new_table_{day_YYYYMMDD}",
        ...,
    )

You can also nest them!

In [ ]:
for day in bqt.param_range("day", "2018-01-01", "2018-01-10"):
    for country in bqt.param_range("country", ["US", "CA", "BR"]):
        bqt.create_table(
            # this will run for every combination of date and country
            "SELECT * FROM my_table WHERE day = '{day}' AND country = '{country}'",
            # also works in the table name, this will create one partition per day
            "my_new_table_{day_YYYYMMDD}",
            ...,
        )

Want more control?

In [ ]:
for day in bqt.param_range("day", "2018-01-01", "2018-07-01", date_part="months", step_size=2):
    # this will run for every other month from 2018-01-01 to 2018-07-01
    # step_size = 2 means it jumps 2 steps on each iteration
    # date_part = 'months' means it'll jump by months rather than days
    pass